In [1]:
import sys
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
import os
import cv2

%matplotlib inline

In [11]:
img_dire = r'../PapSmearData/smear2005/New database pictures'
#read cell image
#read cell image
Directories = os.listdir(img_dire)
# print(Directories)
# imgs = np.ndarray(shape=(781,224,224,3))

transformer = transforms.Compose([transforms.ToTensor(),
                                  transforms.Resize((224,224)),
#                                   transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5)),
                                  transforms.RandomRotation(degrees=(-180, 180))])

imgs = []
label=[]
for d in Directories:
    files_name = os.listdir(img_dire + '/' + d)
    for file in files_name:
        if file[-1] == 'P':
            file_path = img_dire + '/' + d + '/' + file
#             img = Image.open(file_path)
            img = cv2.imread(file_path)
            img_new = cv2.resize(img,dsize=(224,224),interpolation=cv2.INTER_LINEAR)
#             img_new = transformer(img_new)
            img_new = img_new[:,:,[2,1,0]]#BGR to RGB
            
            img_new = np.transpose(img_new, (2,0,1))
           
#             np.append(imgs, img_new)
            imgs.append(img_new)
            if d == 'carcinoma_in_situ':
                label.append(6)
            elif d == 'light_dysplastic':
                label.append(3)
            elif d == 'moderate_dysplastic':
                label.append(4)
            elif d == 'severe_dysplastic':
                label.append(5)
            elif d == 'normal_columnar':
                label.append(2)
            elif d == 'normal_intermediate':
                label.append(1)
            elif d == 'normal_superficiel':
                label.append(0)

In [12]:
len(label),len(imgs)

(917, 917)

In [13]:
imgs_arr = np.array(imgs)

# imgs[1].shape
imgs_train = torch.Tensor(imgs_arr)
# type(imgs)
y = torch.Tensor(np.array(label))

In [14]:
type(imgs_train), imgs_train.size(), y.size()

(torch.Tensor, torch.Size([917, 3, 224, 224]), torch.Size([917]))

In [15]:
#dataset
from torch.utils.data import DataLoader, TensorDataset
smear = TensorDataset(imgs_train, y)

In [17]:
import torchvision.transforms as transforms
# transformer = transforms.Compose([transforms.ToTensor(),
#                                  transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5)),
#                                  transforms.RandomRotation(degrees=(-180, 180))])
train_loader = DataLoader(dataset = smear, batch_size=32, shuffle = True)

In [19]:
EPOCH = 1
CLASSES = 7
LR = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [40]:
from torchvision.models import resnet18

model = resnet18(pretrained=True)


In [41]:
for param in model.parameters():
    param.requires_grad = False

fc_features = model.fc.in_features
model.fc = torch.nn.Linear(fc_features, 7)

creterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=LR, momentum=0.9)
model.to(DEVICE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [42]:

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
#         print(type(target))
        target = target.long()
        optimizer.zero_grad()
        output = model(data)
        loss = creterion(output, target)
        loss.backward()
        optimizer.step()

In [31]:
DEVICE

device(type='cuda')

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [43]:
train(model, DEVICE, train_loader, optimizer, EPOCH)

In [32]:
torch.cuda.is_available()

True